In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import tqdm
import tensorflow as tf
from tensorflow import keras
from functools import partial

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [17]:
X_test
y_test

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [88]:
import cv2
import glob
import sys
import os
import numpy as np
import tqdm
from sklearn.model_selection import train_test_split

def load_file(path,x,y,x_1,y_1):
    files = glob.glob(path+'./*.jpg')
    if not files:
        print("Chck Phath")
        sys.exit()
    img = []
    for index, i in enumerate(tqdm.tqdm((files))):
        im = cv2.imread(i, cv2.IMREAD_GRAYSCALE)[y:y_1, x:x_1]
        if index == 0:
            ref_img = im
        img.append(np.stack([im, ref_img], axis = 2))
    return np.array(img)

features = load_file('data/', 100,100,200,200)
target = np.array(pd.read_csv('data/target.csv', encoding='CP949'))[:5,3]
X_train,  X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.33, random_state=0)
X_mean = X_train.mean(keepdims=True)
X_std = X_train.std(keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std

100%|██████████| 5/5 [00:00<00:00, 143.24it/s]


In [95]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[100, 100, 2]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='sigmoid'),
])

In [96]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 100, 100, 64)      6336      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 50, 50, 128)       147584    
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 25, 25, 256)       295168    
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 25, 25, 256)      

In [98]:
model.compile(loss="mse", optimizer="RMSprop", metrics=["mse"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model.evaluate(X_valid, y_valid)

Epoch 1/10
1/1 [==============================] - 0s 157ms/step - loss: 0.2503 - mse: 0.2503 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 2/10
1/1 [==============================] - 0s 36ms/step - loss: 0.1459 - mse: 0.1459 - val_loss: 9.9882e-04 - val_mse: 9.9882e-04
Epoch 3/10
1/1 [==============================] - 0s 37ms/step - loss: 0.1192 - mse: 0.1192 - val_loss: 9.9882e-04 - val_mse: 9.9882e-04
Epoch 4/10
1/1 [==============================] - 0s 38ms/step - loss: 0.2298 - mse: 0.2298 - val_loss: 9.9882e-04 - val_mse: 9.9882e-04
Epoch 5/10
1/1 [==============================] - 0s 39ms/step - loss: 0.1185 - mse: 0.1185 - val_loss: 0.0010 - val_mse: 0.0010
Epoch 6/10
1/1 [==============================] - 0s 37ms/step - loss: 0.0680 - mse: 0.0680 - val_loss: 9.9882e-04 - val_mse: 9.9882e-04
Epoch 7/10
1/1 [============

In [99]:
score

[0.000998801551759243, 0.000998801551759243]

In [86]:
X_train.shape

(3, 100, 100, 2)

In [89]:
X_valid.shape

(2, 100, 100, 2)